# saving data to disk

In [1]:
# https://stackoverflow.com/questions/25980018/importerror-hdfstore-requires-pytables-no-module-named-tables
!pip install tables

  Using cached https://files.pythonhosted.org/packages/ab/79/4e1301a87f3b7f27aa6c9cb1aeba4875ff3edb62a6fe3872dc8f04983db4/tables-3.5.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e6/35/f187bdf23be87092bd0f1200d43d23076cee4d0dec109f195173fd3ebc79/mock-2.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/14/09/12fe9a14237a6b7e0ba3a8d6fcf254bf4b10ec56a0185f73d651145e9222/pbr-5.1.3-py2.py3-none-any.whl


In [2]:
import os
import sys
print(sys.version)
import h5py
import pandas
print('pandas',pandas.__version__)
import numpy
print('numpy',numpy.__version__)
import sqlite3
print('sqlite3',sqlite3.version)
import pickle
from faker import Faker
fake = Faker()
import time

3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]
pandas 0.23.4
numpy 1.13.3
sqlite3 2.6.0


# create a couple dataframes to save

In [3]:
row_count=1000000
df_numeric = pandas.DataFrame(numpy.random.randint(0,1000,
                      size=(row_count, 4)), 
                      columns=list('ABCD'))

print(df_numeric.shape)

df_numeric.head()

(1000000, 4)


,A,B,C,D
0,890,539,167,836
1,662,278,482,730
2,62,201,204,770
3,323,883,849,970
4,89,323,926,963


In [4]:
start_time=time.time()
list_of_dicts=[]
num_rows=8500 # 850 is ~1MB CSV and takes 1.5 seconds; 8500 takes 15 seconds
for indx in range(num_rows):
    list_of_dicts.append({'name':fake.name(),
                    'date':fake.date(),
                    'domain name':fake.domain_name(),
                    'day of month':fake.day_of_month(),
                    'day of week':fake.day_of_week(),
                    'country':fake.country(),
                    'company':fake.company(),
                    'city':fake.city(),
                    'email':fake.ascii_email(),
                    'bank':fake.bank_country()})
    
df_text = pandas.DataFrame(list_of_dicts)

print('elapsed',round(time.time()-start_time,2),'seconds')
df_text.head()

elapsed 20.32 seconds


,bank,city,company,country,date,day of month,day of week,domain name,email,name
0,GB,Paigeport,Johnson-Shaw,Burkina Faso,1994-10-14,02,Saturday,garcia.com,pwong@rodriguez.info,Daniel Bartlett
1,GB,West Phillipfurt,Ramirez-Neal,Heard Island and McDonald Islands,2011-05-08,03,Monday,hicks-moran.com,kwarren@gmail.com,Angela Johnson
2,GB,New Cheryl,Phillips-Guzman,Argentina,1971-09-21,15,Saturday,woods.com,stevenchung@hotmail.com,Michael White
3,GB,Port Anthony,"Pena, Berry and Villa",Antigua and Barbuda,1988-12-12,02,Monday,lynn-payne.com,andrew67@lee-terrell.com,Eugene Hernandez
4,GB,Gilbertside,Morales-Smith,Kuwait,2010-08-11,23,Wednesday,krause.com,brobinson@flynn.com,Kelly Young


# compare size of disk for single dataframe 

## HDF5
HDF overview:

https://en.wikipedia.org/wiki/Hierarchical_Data_Format

Python package:

https://www.h5py.org/<BR>
http://docs.h5py.org/en/stable/

Pandas integration:

https://glowingpython.blogspot.com/2014/08/quick-hdf5-with-pandas.html<BR>
https://stackoverflow.com/questions/28170623/how-to-read-hdf5-files-in-python<BR>
https://medium.com/@jerilkuriakose/using-hdf5-with-python-6c5242d08773<BR>
https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html

In [5]:
# https://stackoverflow.com/questions/41173254/how-should-i-use-h5py-lib-for-storing-time-series-data

with pandas.HDFStore('temp.h5', 'w') as h:
    df_numeric.to_hdf(h, 'temp') # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_hdf.html

In [6]:
print(os.path.getsize("temp.h5")/(1024*1024),'MB file on disk')

38.15387725830078 MB file on disk


to read the content, use

In [7]:
df_out = pandas.read_hdf('temp.h5', 'temp')

## compare to CSV on disk

In [8]:
df_numeric.to_csv('temp.csv')

In [9]:
print(os.path.getsize("temp.csv")/(1024*1024),'MB file on disk')

21.40812110900879 MB file on disk


## compare to SQLite

https://www.dataquest.io/blog/python-pandas-databases/<BR>
https://stackoverflow.com/questions/14431646/how-to-write-pandas-dataframe-to-sqlite-with-index<BR>
https://pythonspot.com/sqlite-database-with-pandas/<BR>
https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html<BR>
http://sdsawtelle.github.io/blog/output/large-data-files-pandas-sqlite.html

In [10]:
conn = sqlite3.connect("temp.db")
cur = conn.cursor() # https://docs.python.org/3/library/sqlite3.html#cursor-objects

In [11]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html

df_numeric.to_sql(name="data", con=conn, if_exists="append", index=False)

In [12]:
print(os.path.getsize("temp.db")/(1024*1024),'MB file on disk')

74.859375 MB file on disk


## Python pickle serialization

In [13]:
df_numeric.to_pickle("temp.pkl")

In [14]:
print(os.path.getsize("temp.pkl")/(1024*1024),'MB file on disk')

30.518251419067383 MB file on disk


# save two dataframes to file

## HDF5

https://datascience.stackexchange.com/questions/33171/what-s-the-best-way-to-save-many-pandas-dataframes-together

In [15]:
h5_fout = h5py.File('temp.h5')

h5_fout.create_dataset(
        name='numeric',
        data=df_numeric,
        compression='gzip', compression_opts=4)

h5_fout.create_dataset(
        name='text',
        data=df_text,
        compression='gzip', compression_opts=4,
        dtype=h5py.special_dtype(vlen=str)) # http://docs.h5py.org/en/stable/special.html

h5_fout.create_dataset('description', data='some dataframes')
h5_fout.close()

## Python pickle serialization

In [16]:
with open('temp.pkl', "wb") as f:
    pickle.dump(df_text, f)
    pickle.dump(df_numeric, f)

In [17]:
print(os.path.getsize("temp.pkl")/(1024*1024),'MB file on disk')

32.00189971923828 MB file on disk


In [18]:
# https://stackoverflow.com/questions/20716812/saving-and-loading-multiple-objects-in-pickle-file
def loadall(filename):
    with open(filename, "rb") as f:
        while True:
            try:
                yield pickle.load(f)
            except EOFError:
                break

In [19]:
items = list(loadall('temp.pkl'))

In [20]:
len(items)

2

In [21]:
items[0].head()

,bank,city,company,country,date,day of month,day of week,domain name,email,name
0,GB,Paigeport,Johnson-Shaw,Burkina Faso,1994-10-14,02,Saturday,garcia.com,pwong@rodriguez.info,Daniel Bartlett
1,GB,West Phillipfurt,Ramirez-Neal,Heard Island and McDonald Islands,2011-05-08,03,Monday,hicks-moran.com,kwarren@gmail.com,Angela Johnson
2,GB,New Cheryl,Phillips-Guzman,Argentina,1971-09-21,15,Saturday,woods.com,stevenchung@hotmail.com,Michael White
3,GB,Port Anthony,"Pena, Berry and Villa",Antigua and Barbuda,1988-12-12,02,Monday,lynn-payne.com,andrew67@lee-terrell.com,Eugene Hernandez
4,GB,Gilbertside,Morales-Smith,Kuwait,2010-08-11,23,Wednesday,krause.com,brobinson@flynn.com,Kelly Young


In [22]:
items[1].head()

,A,B,C,D
0,890,539,167,836
1,662,278,482,730
2,62,201,204,770
3,323,883,849,970
4,89,323,926,963
